# **Polars en Python: Trabajando con Varios DataFrames y Fechas**

En este cuaderno replicamos el flujo del notebook de *Pandas* (cargar varios datasets, hacer *joins* y trabajar con fechas), pero usando **Polars**.


### **Instalar e importar librería**

In [1]:
!pip -q install polars

import polars as pl

### **Pandas vs Polars: diferencias clave**

- **Motor y rendimiento**: Pandas ejecuta operaciones sobre estructuras en memoria (en muchas operaciones, sin paralelizar). Polars está escrito en **Rust** y suele ejecutar muchas operaciones en **paralelo**.
- **Eager vs Lazy**: Pandas es (casi siempre) *eager* (ejecuta al momento). Polars tiene modo *eager* **y** un modo **lazy** que permite optimizaciones (por ejemplo, *pushdown* de filtros/selecciones y optimización del plan de ejecución).
- **API**: Pandas se apoya mucho en *indexing* y en `apply` fila a fila. Polars está orientado a **expresiones** (`pl.col(...)`), lo que favorece operaciones vectorizadas y planes optimizables.
- **Tipos de datos**: Polars tiende a ser más estricto y explícito con los tipos (útil para evitar sorpresas).
- **UDFs (apply)**: En Pandas es común usar `apply`. En Polars se recomienda priorizar expresiones nativas (más rápidas). Las funciones Python existen (`map_elements`), pero suelen ser más lentas y pueden romper optimizaciones.

> Regla práctica: para transformaciones grandes, joins y agregaciones (ETL), **Polars** suele brillar; para trabajo exploratorio y compatibilidad con el ecosistema clásico, **Pandas** sigue siendo una gran opción.


### **Cargar Varios DataFrames**

In [2]:
df1 = pl.read_csv('./data/01_dataset.csv')

df2 = pl.read_csv('./data/Sistemas Big Data/02_dataset.csv')

df3 = pl.read_csv('./data/03_dataset.csv')

FileNotFoundError: No such file or directory (os error 2): ./data/Sistemas Big Data/02_dataset.csv

### **Mostrar los 5 primeros elementos de los df**

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df3.head()


# **JOINS**

### **Unir el df1 con df2**

In [ ]:
df_con_df1_df2 = df1.join(df2, on='user', how='left')

df_con_df1_df2.head()

### **Unir el anterior Df con df3**

In [ ]:
df_final = df_con_df1_df2.join(df3, on='ciudad', how='left')

df_final.head()

### **Mostrar los tipos de dataframe**

In [ ]:
# En Polars puedes ver el esquema (nombre de columna -> tipo)

df_final.schema

# **FECHAS**

### **Trasnformar a Fehca la columna fecha_publicado.**

In [ ]:
# Las fechas por defecto suelen venir como cadenas de texto.
# En Polars las convertimos con str.strptime.
# Nota: mantenemos el nombre con el mismo typo del notebook original para que los pasos siguientes funcionen igual.

df_final = df_final.with_columns(
    pl.col('fecha_publicado')
      .str.strptime(pl.Date, '%d/%m/%Y', strict=False)
      .alias('fecha_pulblicacion_norm')
)

df_final.head()

### **Crear un columna con el dia de publicación**

In [ ]:
df_final = df_final.with_columns(
    pl.col('fecha_pulblicacion_norm').dt.strftime('%A').alias('dia_semana')
)

df_final.head()

In [ ]:
# Ejemplo usando funciones (UDF): existe, pero normalmente es más lento que expresiones nativas.

def darDiaSemana(fecha):
    return fecha.strftime('%A') if fecha is not None else None

df_final = df_final.with_columns(
    pl.col('fecha_pulblicacion_norm').map_elements(darDiaSemana).alias('dia_semana_fun')
)

df_final.head()

### **Crear un columna con el mes y año de publicación**

In [ ]:
df_final = df_final.with_columns(
    pl.col('fecha_pulblicacion_norm').dt.strftime('%m/%Y').alias('mes')
)

df_final.head()

### **Sacar de la columna fecha_publicado el mes y año**

In [ ]:
# Igual que en Pandas (x[3:]) pero con slicing de strings en Polars (0-index)

df_final = df_final.with_columns(
    pl.col('fecha_publicado').str.slice(3).alias('mes')
)

df_final.head()

In [ ]:
# Usando función (split)

def extraer_mes_y_anio_de_cadena(fecha_str):
    if fecha_str is None:
        return None
    partes = fecha_str.split('/')
    mes = partes[1]
    anio = partes[2]
    return mes + '/' + anio

# Creamos una columna auxiliar para ver el resultado

df_final = df_final.with_columns(
    pl.col('fecha_publicado').map_elements(extraer_mes_y_anio_de_cadena).alias('mes_split_fun')
)

df_final.head()

In [ ]:
# Funcion usando slice

def extraer_mes_y_anio_con_slice(fecha_str):
    if fecha_str is None:
        return None
    mes = fecha_str[3:5]
    anio = fecha_str[6:10]
    return mes + '/' + anio

# Creamos una columna auxiliar para ver el resultado

df_final = df_final.with_columns(
    pl.col('fecha_publicado').map_elements(extraer_mes_y_anio_con_slice).alias('mes_slice_fun')
)

df_final.head()

# **Guardar nuestros datos trasformados a CSV, JSON, SQL, ...**

In [ ]:
# Guardar a CSV

df_final.write_csv('./data/df_final_polars.csv')